In [2]:
import pandas as pd
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler

In [3]:
# Cargar datos preprocesados
covid_data = pd.read_csv("../covid19_clean_data.csv", index_col=0)
covid_data.shape

(6349701, 9)

In [4]:
# Separar las caracteristicas de las clases 
X = covid_data.drop("estado", axis=1)
y = covid_data["estado"]

In [5]:
# Crear datasets de entrenamiento(80%) y validación(20%). 
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=42
)

In [6]:
# Definición de variables categoricas a usar en el modelo
    # Se excluye la variable de municipio pues el modelo considera la variable no relevante
categorical_features = ['sexo', 'tipo_contagio', 'rango_edad','asintomatico']#,'departamento','municipio'

# Definición de variables númericas a usar en el modelo
    # Excluye asintomatico pues el modelo considera la variable no relevante
    # Excluye id_caso pues no infiere en la clasificacón del estado
numerical_features = ['edad'] 

# Preprocesamiento de las caracteristicas categoricas y númericas definidas previamente
preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)

# Creación de flujo de trabajo que seguirá el modelo

In [7]:
pipeline = ImbPipeline(
   [
       ('undersampling', RandomUnderSampler(sampling_strategy = 'majority', random_state=42)),
       ("preprocessor", preprocessor),
       ("classifier", HistGradientBoostingClassifier(random_state=42))
   ],
   verbose=True
)

In [8]:
grid =  GridSearchCV(
    pipeline,
    param_grid = { 
        'classifier__max_iter': [50, 100, 150],
        'classifier__learning_rate': [0.5, 0.1, 0.01], 
        'classifier__max_depth': [3, 9, None]}, 
    n_jobs=-1,
    cv=3,
    error_score='raise'
    #iverbose=4
)

In [9]:
# Fit the grid on the training data
grid.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 3) Processing undersampling, total=  27.1s
[Pipeline] ...... (step 2 of 3) Processing preprocessor, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   1.3s


GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('undersampling',
                                        RandomUnderSampler(random_state=42,
                                                           sampling_strategy='majority')),
                                       ('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['sexo',
                                                                          'tipo_contagio',
                                                                          'rango_edad',
                                                                          'asintomatico']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['edad'])])),
                                       ('classifier',
                                        HistGradientBoostingClassifier(random_state=42))],
                                verbose=True),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [0.5, 0.1, 0.01],
                         'classifier__max_depth': [3, 9, None],
                         'classifier__max_iter': [50, 100, 150]})

In [10]:
# Save pipeline for future use without fitting
joblib.dump(grid, 'HistGradientBoosting_GridSearch_RandomUnderSampling.joblib')

['HistGradientBoosting_GridSearch_RandomUnderSampling.joblib']

In [11]:
grid.predict(X_test)

array(['Fallecido', 'Fallecido', 'Fallecido', ..., 'Fallecido',
       'Fallecido', 'Moderado'], dtype=object)

In [12]:
# Best parameters
print(grid.cv_results_)
print(grid.best_estimator_)

{'mean_fit_time': array([26.31635038, 30.10699209, 29.68240094, 26.77239362, 26.76172988,
       25.2817142 , 24.00663471, 23.92673683, 24.59728098, 24.78798143,
       25.26386062, 25.52958886, 26.80892682, 26.25761819, 26.37750856,
       25.62168447, 26.53434396, 26.34913365, 27.39062492, 30.01679023,
       31.93009027, 28.21038834, 33.45574236, 34.88227638, 25.75454553,
       32.77542392, 32.42560379]), 'std_fit_time': array([0.48920677, 1.47372073, 1.13935519, 0.74617087, 0.96859669,
       0.6391755 , 0.75598043, 0.55490168, 0.68294261, 1.45080622,
       0.93762803, 0.60032812, 1.02882223, 1.20016757, 0.81576379,
       0.87992149, 0.57896318, 1.25372647, 0.52259273, 0.65108204,
       0.76359603, 0.11475504, 0.14407271, 0.53541967, 0.81208403,
       0.61914737, 0.61835207]), 'mean_score_time': array([ 19.12253753,  18.43626801,  17.33181159,  20.43014852,
        19.40260975,  18.09056536,  18.4674449 ,  19.05190372,
        21.21663745,  15.09522645,  15.06806588,  15.81612

In [13]:
# Fit the tuned_pipeline on the training data
tuned_pipeline = grid.best_estimator_.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 3) Processing undersampling, total=  26.2s
[Pipeline] ...... (step 2 of 3) Processing preprocessor, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   1.0s


In [14]:
# Predict on the test set
y_pred = tuned_pipeline.predict(X_test)

In [15]:
# Generate classification report
report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

   Fallecido       0.03      0.94      0.05     28487
       Grave       0.00      0.04      0.00        23
        Leve       0.99      0.04      0.07   1241208
    Moderado       0.00      0.28      0.00       223

    accuracy                           0.06   1269941
   macro avg       0.25      0.33      0.03   1269941
weighted avg       0.97      0.06      0.07   1269941

